In [ ]:
import torch
import transformers
from transformers import AutoTokenizer

import pandas as pd

mcrae_train25_file = "/home/amitgajbhiye/cardiff_work/llm_embeddings/data/mcrae/most_frequent_properties/train25.tsv"
train_df = pd.read_csv(mcrae_train25_file, sep="\t", names=["concept", "property", "label"])

positive_train_df = train_df[train_df["label"] == 1]

positive_train_df.sort_values(by=["property"], inplace=True)
positive_train_df.reset_index(inplace=True, drop=True)

print ("positive_train_df")
print (positive_train_df)

uniq_props = positive_train_df["property"].unique()

print ("uniq_props")
print (uniq_props)

num_concepts = 5

all_concepts_list = []

for prop in uniq_props:
    concepts = positive_train_df[positive_train_df["property"] == prop]["concept"].to_list()[:num_concepts]
    concepts_list = ", ".join(concepts)
    
    all_concepts_list.append((concepts_list, prop))
    
    print (concepts_list)
    
    

prompt = f"Enumerate the five most salient properties shared by the following concepts - <CONCEPT_LIST>. Generate only the numbered list of properties."

prompt_list = [(prompt.replace("<CONCEPT_LIST>", concept_list), original_property) for concept_list, original_property in all_concepts_list]

print (prompt_list)

In [1]:
import torch
import transformers
from transformers import AutoTokenizer
import random

In [2]:

# model = "meta-llama/Llama-2-13b-chat-hf"
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
input_file = "/home/amitgajbhiye/cardiff_work/llm_embeddings/data/ufet/clean_types.txt"
# df = pd.read_csv(input_file, sep="\t", names=["type"])

with open(input_file, "r") as f:
    types = f.readlines()

types = [type.strip("\n").strip() for type in types][0:10]
print(f"num_types: {len(types)}")
print(f"sample_types: {random.sample(types, 5)}")

num_types: 10
sample_types: ['event', 'time', 'accident', 'person', 'place']


In [4]:
prompt = f"List the five most salient properties of <CONCEPT>. Generate only the numbered list of properties."
prompts = [prompt.replace('<CONCEPT>', type) for type in types][0:150]

print (prompts)


['List the five most salient properties of person. Generate only the numbered list of properties.', 'List the five most salient properties of group. Generate only the numbered list of properties.', 'List the five most salient properties of organization. Generate only the numbered list of properties.', 'List the five most salient properties of location. Generate only the numbered list of properties.', 'List the five most salient properties of entity. Generate only the numbered list of properties.', 'List the five most salient properties of time. Generate only the numbered list of properties.', 'List the five most salient properties of object. Generate only the numbered list of properties.', 'List the five most salient properties of event. Generate only the numbered list of properties.', 'List the five most salient properties of place. Generate only the numbered list of properties.', 'List the five most salient properties of accident. Generate only the numbered list of properties.']


In [ ]:
response_list = []

file_name = f'generated_properties_{model.replace("/", "_").replace("-", "_")}.txt'

with open (file_name, 'w') as out_file:
    for prompt in prompts:
        
        sequences = pipeline(
            prompt,
            do_sample=True,
            top_k=1,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            max_length=350,
          )
        
        for seq in sequences:
            response_list.append(f"{seq['generated_text']}\n\n")
            print(f"{seq['generated_text']}")

            out_file.write(f'{seq["generated_text"]}\n')
            
            print("===================================")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


List the five most salient properties of person. Generate only the numbered list of properties.

1. Consciousness
2. Reason
3. Emotion
4. Morality
5. Sociality
